In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(temperature=0, model="gpt-5-nano")

In [5]:
# 원하는 출력 구조 정의 pydantic
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [6]:
question = "지구 온난화의 심각성에 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

chain.invoke({"question": question})

{'description': '지구 온난화는 평균 기온 상승으로 극한 기상 현상 증가, 해수면 상승, 생태계 위협 등을 초래해 인류의 식량·물 안정과 경제안보에 심각한 영향을 미치고 있습니다. 속도가 빠를수록 위험이 커집니다.',
 'hashtags': '#지구온난화 #기후위기 #환경안전'}

## Pydantic 없이 JSON 형태로 응답받기

In [8]:
model = ChatOpenAI(temperature=0, model="gpt-5-nano")

# 질문
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON Parser
parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요"),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

# 지시사항 주입
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인
chain = prompt | model | parser

response = chain.invoke({"question": question})

print(response)

{'description': '지구 온난화는 지구의 평균 표면 온도가 장기적으로 상승하는 현상으로, 주로 인간의 활동으로 방출되는 온실가스가 대기의 열을 가두기 때문입니다. 화석연료의 연소, 산업 공정, 벌목 등으로 이산화탄소(CO2), 메탄(CH4), 아산화질소(N2O) 등의 온실가스 농도가 높아지면서 지구가 더 많은 열을 보유하게 됩니다. 자연적 기후 변동도 영향을 주지만, 산업화 이후의 온도 상승은 주로 인류 활동에 의해 주도됩니다. 이로 인해 여름의 폭염 빈도와 강도 증가, 가뭄과 홍수의 불규칙성, 해수면 상승, 빙하 및 해빙의 축소 등 다양한 영향이 나타나고 있습니다. 대응책으로는 재생에너지 확대와 에너지 효율 개선을 통한 에너지 전환, 교통과 산업의 탈탄소화, 산림 보전 및 재조림, 탄소 포집 및 저장(CCS), 탄소발자국 감소 및 기후 적응 정책이 필요합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#탄소발자국', '#재생에너지', '#에너지전환', '#탄소중립', '#기후적응']}
